In [ ]:
import numpy as np
import tensorflow as tf
import datetime
import data_helpers
from sent_cnn import SentCNN
%load_ext autoreload
%autoreload 2

In [ ]:
x_u, x_r, y, max_len = data_helpers.get_training_examples_for_softmax("./data/webquestions.examples.train.e2e.top10.filter.patrel.sid.tsv")

In [ ]:
pad = lambda x: data_helpers.pad_sentences(x, max_len)
pad_lst = lambda x: map(pad, x)
x_u = map(pad, x_u)
x_r = map(pad_lst, x_r)

In [ ]:
tokens, U = data_helpers.get_pretrained_wordvec_from_file("./data/word_representations/glove.6B.100d.txt", (400000, 100))

In [ ]:
dctize = lambda word: tokens[word] if tokens.has_key(word) else tokens["pad"]
dctizes = lambda words: map(dctize, words)
dctizess = lambda wordss: map(dctizes, wordss)
x_u_i = np.array(map(dctizes, x_u))
x_r_i = np.array(map(dctizess, x_r))
y = np.array(y)

In [ ]:
cnn = SentCNN(sequence_length=max_len, 
              num_classes=6, 
              init_embeddings=U, 
              filter_sizes=[3, 4], 
              num_filters=3,
              embeddings_trainable=False)
batch_size = 500
global_step = tf.Variable(0, name="global_step", trainable=True)
optimizer = tf.train.AdamOptimizer()
grads_and_vars = optimizer.compute_gradients(cnn.loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for _ in range(400000):
        indices = np.random.choice(len(x_u_i), batch_size)
        x_u_batch = x_u_i[indices]
        x_r_batch = x_r_i[indices]
        y_batch = y[indices]
        feed_dict = {
            cnn.input_x_u: x_u_batch, 
            cnn.input_x_r: x_r_batch,
            cnn.input_y: y_batch
        }
    
        _, step, loss, accuracy, cosine, pred = sess.run(
            [train_op, global_step, cnn.loss, cnn.accuracy, cnn.cosine, cnn.predictions], feed_dict)
        time_str = datetime.datetime.now().isoformat()
        if step % 50 == 0:
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))  
            print pred[:5], y_batch[:5]

In [ ]:
np.shape(x_r_i)

In [1]:
import numpy as np
import tensorflow as tf
import datetime
import data_helpers
import train_cnn
from sent_cnn import SentCNN
%load_ext autoreload
%autoreload 2

In [2]:
from config import config

In [5]:
x_u_i, x_r_i, y, max_len, U = train_cnn.load_data(config)

In [8]:
train_cnn.train_cnn(x_u_i, x_r_i, y, max_len, U, config)

DEBUG: embedded_u -> Tensor("embedding_1/embedding_lookup:0", shape=(?, 22, 100), dtype=float32)
DEBUG: embedded_r -> Tensor("embedding_1/embedding_lookup_1:0", shape=(?, 6, 22, 100), dtype=float32)
DEBUG: embedded_u_expanded -> Tensor("embedding_1/ExpandDims:0", shape=(?, 22, 100, 1), dtype=float32)
DEBUG: embedded_r_expanded -> Tensor("embedding_1/ExpandDims_1:0", shape=(?, 6, 22, 100, 1), dtype=float32)
DEBUG: pooled_outputs_u -> [<tf.Tensor 'conv-maxpool-3-u_1/pool-u:0' shape=(?, 1, 1, 3) dtype=float32>, <tf.Tensor 'conv-maxpool-4-u_1/pool-u:0' shape=(?, 1, 1, 3) dtype=float32>]
DEBUG: h_pool_u -> Tensor("concat_2:0", shape=(?, 1, 1, 6), dtype=float32)
DEBUG: h_pool_flat_u -> Tensor("Reshape_2:0", shape=(?, 1, 6), dtype=float32)
DEBUG: pooled_outputs_r -> [<tf.Tensor 'conv-maxpool-3-u_1/concat:0' shape=(?, 1, 6, 3) dtype=float32>, <tf.Tensor 'conv-maxpool-4-u_1/concat:0' shape=(?, 1, 6, 3) dtype=float32>]
DEBUG: h_pool_r -> Tensor("concat_3:0", shape=(?, 1, 6, 6), dtype=float32)
DE

KeyboardInterrupt: 